<a href="https://colab.research.google.com/github/sonisaransh02/AllinSemanticSearch/blob/main/allinpod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/dataAllIn

/content/drive/MyDrive/dataAllIn


In [79]:
import os
from glob import glob

from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import re

In [ ]:
data_path = '/content/drive/MyDrive/dataAllIn/data/*'
html_files = glob(data_path)
len(html_files)

In [ ]:
html_files[0]

In [7]:
arr = []
for file in tqdm(html_files):
    soup = BeautifulSoup(open(file,'rb'), 'html.parser')
    
    speaker = []
    text = []
    timestamp = []
    episode = []

    for each in soup.find_all('div',attrs={'class':"transcript is-main"}):
        speaker.append(each.find('li', attrs={'class':'speaker'}).text)
        text.append(each.find('mark').text)
        timestamp.append(each.find('li', attrs={'class':'tg-caption', 'id':'playTranscript'}).text)
        episode.append(file)
    

    arr.append(pd.DataFrame({
            'time' : timestamp,
            'text' : text,
            'speaker' : speaker,
            'episode' : episode
        }))

100%|██████████| 106/106 [01:42<00:00,  1.04it/s]


In [ ]:
data = pd.concat(arr).reset_index(drop=True)
#regex for only episode number.
data['episode']=  data['episode'].str.extract('E(\d*\.?\d*)_')
data = data.sort_values(by=['episode','time'])
data 

In [ ]:
sentences = data.text.tolist()
print(sentences)

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
!pip install transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

In [54]:
embeddings = model.encode(sentences)

In [ ]:
query=["theranos"]
queryEmbedding=model.encode(query)
print(queryEmbedding)

In [56]:
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
sims = cosine_similarity(embeddings, queryEmbedding) 

In [ ]:
sims.shape

In [ ]:
data['similarity'] = sims
data

In [ ]:
data.sort_values(by='similarity',ascending=False)

In [91]:
data.sort_values(by='similarity',ascending=False).iloc[0]['text']

'with like you end up with a magic leap or Theranos, Theranos'